In [1]:
import os, glob
import json

import numpy as np
import pandas as pd


In [11]:
lookup_path = "/home/craig.car/repos/chiron/chironata/data/cts_lookup_table.json"
with open(lookup_path) as f:
    lookup = json.load(f)

lookup["urn:cts:greekLit:tlg0640.tlg001"]

['alciphron_1896']

In [10]:
for path in ["/scratch/craig.car/english_trans-dev/tlg0640.tlg001.ogl-eng1.xml"]:
    prefix = os.path.splitext(path)[0]
    print(prefix)

/scratch/craig.car/english_trans-dev/tlg0640.tlg001.ogl-eng1


In [2]:
text_ctsurn = []
num_transl = []
transl_langs = []
num_src_sents = []
text_top_1s = []
text_top_5s = []
text_top_10s = []

not_transl_keys = ["num_src_sents", "num_translations", "text_top1", "text_top5", "text_top10"]

eval_dicts_dir = "/home/craig.car/repos/chiron/chironata/eval_datafiles/"
for eval_json_path in glob.iglob(eval_dicts_dir+"*.json"):
    ctsurn = os.path.splitext(os.path.basename(eval_json_path))[0]
    with open(eval_json_path) as f:
        eval_json = json.load(f)
    text_ctsurn.append(ctsurn)
    # print(eval_json["apolloniusRhodius_1_1791"].keys())
    num_transl.append(eval_json["num_translations"])
    num_src_sents.append(eval_json["num_src_sents"])
    text_top_1s.append(eval_json["text_top1"])
    text_top_5s.append(eval_json["text_top5"])
    text_top_10s.append(eval_json["text_top10"])
    text_langs = set()
    for key in eval_json.keys():
        if key not in not_transl_keys:
            text_langs.add(eval_json[key]["tgt_lang"])
    transl_langs.append(text_langs)


In [3]:
print(len(text_ctsurn))
print(len(num_transl))
print(len(transl_langs))
print(len(num_src_sents))
print(len(text_top_1s))
print(len(text_top_5s))
print(len(text_top_10s))


709
709
709
709
709
709
709


In [4]:
eval_df = pd.DataFrame(text_ctsurn)
eval_df.columns = ["ctsurn"]

In [5]:
eval_df["number_translations"] = num_transl
eval_df["translation_languages"] = transl_langs
eval_df["number_src_sents"] = num_src_sents
eval_df["text_wide_recall@1"] = text_top_1s
eval_df["text_wide_recall@15"] = text_top_5s
eval_df["text_wide_recall@10"] = text_top_10s

In [6]:
eval_df.head()

,ctsurn,number_translations,translation_languages,number_src_sents,text_wide_recall@1,text_wide_recall@15,text_wide_recall@10
0,urn:cts:greekLit:tlg0001.tlg001,3,"{en, fr, it}",1738,0.178941,0.296318,0.353471
1,urn:cts:greekLit:tlg0003.tlg001,7,"{en, fr, de}",4419,0.130379,0.194194,0.224873
2,urn:cts:greekLit:tlg0004.tlg001,1,{en},6440,0.435714,0.654193,0.736801
3,urn:cts:greekLit:tlg0005.tlg001,5,"{en, fr, de}",549,0.234244,0.409107,0.499089
4,urn:cts:greekLit:tlg0005.tlg002,5,"{en, fr, de}",94,0.123404,0.200000,0.261702


In [7]:
eval_df.sort_values("text_wide_recall@10", ascending=[False])

,ctsurn,number_translations,translation_languages,number_src_sents,text_wide_recall@1,text_wide_recall@15,text_wide_recall@10
661,urn:cts:latinLit:phi1212.phi003,1,{en},392,0.757653,0.989796,1.043367
595,urn:cts:latinLit:phi0474.phi055,1,{en},2011,0.776231,0.945301,0.987568
666,urn:cts:latinLit:phi1348.abo011,2,"{fr, de}",517,0.709865,0.913926,0.984526
433,urn:cts:greekLit:tlg0074.tlg004,1,{fr},282,0.549645,0.872340,0.971631
323,urn:cts:greekLit:tlg0034.tlg001,1,{de},406,0.655172,0.874384,0.963054
...,...,...,...,...,...,...,...
463,urn:cts:greekLit:tlg0096.tlg002,1,{de},6062,0.000660,0.006434,0.012042
596,urn:cts:latinLit:phi0474.phi056,2,{de},6913,0.001374,0.004484,0.008969
582,urn:cts:latinLit:phi0474.phi035,2,{de},4047,0.000741,0.005313,0.008772
434,urn:cts:greekLit:tlg0074.tlg008,1,{fr},9,0.000000,0.000000,0.000000
